In [1]:
import mysql.connector
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
USER = "chenriquez"
PASSWORD = "Chilote64#"
IP = "localhost"
BD_NAME = "analitica2"

In [3]:
cnx = mysql.connector.connect(user=USER, 
                                  password=PASSWORD,
                                  host='127.0.0.1',
                                  database=BD_NAME,
                                  auth_plugin='mysql_native_password')

In [4]:
query="select distinct hiercode from rp_main_hiercode_prod_sold"
cursor = cnx.cursor()
cursor.execute(query)    
lst_hiercode=[item[0] for item in cursor.fetchall()]

In [5]:
query2='select hiercode,prodcode, concat(season," ",description) as description  from rp_main_hiercode_prod_sold'
cursor=cnx.cursor()
cursor.execute(query2)
df_input=pd.DataFrame(cursor.fetchall())
field_names = [i[0] for i in cursor.description]
df_input.columns = field_names

In [6]:
print(df_input.shape)

(423754, 3)


In [7]:
def get_matrix(dfiltered):
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
    matrix = tf.fit_transform(dfiltered['description'])
    return linear_kernel(matrix,matrix) 

In [8]:
def get_idx(dfiltered):
    dfiltered.index = pd.RangeIndex(len(dfiltered.index))
    return pd.Series(dfiltered.index, index=dfiltered['prodcode'])

In [9]:
def get_sim(hiercode,prodcode,indeces,cosine_similarities_matrix):
    idx = indeces[prodcode]
    sim_scores = list(enumerate(cosine_similarities_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    list_sim_scores=[list(element) for element in sim_scores]
    list_sim_scores_filtered=list(filter(lambda x: x[0]!=idx,[list(element) for element in sim_scores]))
    sim_scores_list=[  [hiercode,
                        prodcode,
                        indeces[indeces == list(element)[0]].index[0],
                        list(element)[1]] 
                        for element 
                        in list_sim_scores_filtered ]
    sim_scores_list_filtered=list(filter(lambda x:x[3]>=0.5,sim_scores_list))
    df = pd.DataFrame(sim_scores_list_filtered)
    if len(df!=0):
        df.columns=['hiercode','prodcode','prodcode_rel','sim_idx']
    return df

In [ ]:
i=1
for hiercode in lst_hiercode:
    dfiltered=df_input[df_input.hiercode==hiercode]
    dfiltered=dfiltered[['prodcode','description']]
    cosine_similarities_matrix=get_matrix(dfiltered)
    indeces=get_idx(dfiltered)
    print(" Núumero de jerarquía en ejecución{}".format(i))
    i=i+1
    prod_list_exclude=[]
    prod_list=dfiltered['prodcode']
    
    for prodcode in list(prod_list):
        if prodcode in prod_list_exclude:
            pass
        else:
            df=get_sim(hiercode,prodcode,indeces,cosine_similarities_matrix)
            if len(df!=0):
                prod_rel_list=df['prodcode_rel'].tolist()
                prod_rel_list.append(df['prodcode'][0])
                prod_list_exclude.extend(prod_rel_list)
            print(df)    
    
    #if len(df)!=0:
    #    df.to_sql('rp_prod_similarity', cnx, if_exists='append', index=False, chunksize=500 )
    #else:
    #    pass
            

 Núumero de jerarquía en ejecución1
  hiercode       prodcode   prodcode_rel   sim_idx
0  S000065  2000362744275  2000362744282  1.000000
1  S000065  2000362744275  2000362744268  1.000000
2  S000065  2000362744275  2000362744299  0.911695
  hiercode       prodcode   prodcode_rel  sim_idx
0  S000065  2000362744794  2000362744787   0.9078
1  S000065  2000362744794  2000362744770   0.9078
2  S000065  2000362744794  2000362744763   0.9078
  hiercode       prodcode   prodcode_rel  sim_idx
0  S000065  2000362744749  2000362744732   0.9078
1  S000065  2000362744749  2000362744725   0.9078
2  S000065  2000362744749  2000362744718   0.9078
  hiercode       prodcode   prodcode_rel  sim_idx
0  S000065  2000362744695  2000362744688   0.9078
1  S000065  2000362744695  2000362744671   0.9078
2  S000065  2000362744695  2000362744664   0.9078
  hiercode       prodcode   prodcode_rel  sim_idx
0  S000065  2000362744596  2000362744589   0.9078
1  S000065  2000362744596  2000362744572   0.9078
2  S000065

  hiercode       prodcode   prodcode_rel   sim_idx
0  S000065  2000362342075  2000362342051  0.869648
1  S000065  2000362342075  2000362342044  0.869648
2  S000065  2000362342075  2000362342068  0.869648
3  S000065  2000362342075  2000355987054  0.531567
4  S000065  2000362342075  2000355987061  0.531567
5  S000065  2000362342075  2000355987078  0.531567
  hiercode       prodcode   prodcode_rel   sim_idx
0  S000065  2000361976370  2000361976363  0.889465
1  S000065  2000361976370  2000361976356  0.889465
2  S000065  2000361976370  2000361976349  0.889465
   hiercode       prodcode   prodcode_rel   sim_idx
0   S000065  2000346330876  2000346330890  1.000000
1   S000065  2000346330876  2000346330883  1.000000
2   S000065  2000346330876  2000346330906  0.901640
3   S000065  2000346330876  2000337805758  0.727166
4   S000065  2000346330876  2000337805741  0.727166
5   S000065  2000346330876  2000337805734  0.727166
6   S000065  2000346330876  2000337805833  0.724232
7   S000065  2000346330

  hiercode       prodcode   prodcode_rel   sim_idx
0  S000065  2000347061908  2000347061915  1.000000
1  S000065  2000347061908  2000347061892  1.000000
2  S000065  2000347061908  2000347061922  0.623536
  hiercode       prodcode   prodcode_rel   sim_idx
0  S000065  2000362535088  2000362535071  0.859503
1  S000065  2000362535088  2000362535064  0.859503
2  S000065  2000362535088  2000362535057  0.859503
3  S000065  2000362535088  2000362535156  0.501921
4  S000065  2000362535088  2000362535149  0.501921
5  S000065  2000362535088  2000362535132  0.501921
  hiercode       prodcode   prodcode_rel   sim_idx
0  S000065  2000347061854  2000347061861  1.000000
1  S000065  2000347061854  2000347061847  1.000000
2  S000065  2000347061854  2000347061878  0.625221
  hiercode       prodcode   prodcode_rel   sim_idx
0  S000065  2000362535163  2000362535156  0.867237
1  S000065  2000362535163  2000362535149  0.867237
2  S000065  2000362535163  2000362535132  0.867237
3  S000065  2000362535163  2000

  hiercode       prodcode   prodcode_rel   sim_idx
0  S000065  2000361728436  2000361728443  1.000000
1  S000065  2000361728436  2000361728450  1.000000
2  S000065  2000361728436  2000361728467  0.948011
  hiercode       prodcode   prodcode_rel   sim_idx
0  S000065  2000361728481  2000361728498  1.000000
1  S000065  2000361728481  2000361728504  1.000000
2  S000065  2000361728481  2000361728511  0.954428
  hiercode       prodcode   prodcode_rel   sim_idx
0  S000065  2000361728283  2000361728290  1.000000
1  S000065  2000361728283  2000361728306  1.000000
2  S000065  2000361728283  2000361728313  0.936626
  hiercode       prodcode   prodcode_rel  sim_idx
0  S000065  2000361728337  2000361728344  1.00000
1  S000065  2000361728337  2000361728351  1.00000
2  S000065  2000361728337  2000361728368  0.94889
  hiercode       prodcode   prodcode_rel   sim_idx
0  S000065  2000361728382  2000361728399  1.000000
1  S000065  2000361728382  2000361728405  1.000000
2  S000065  2000361728382  20003617

  hiercode       prodcode   prodcode_rel   sim_idx
0  S000065  2000361487104  2000361487111  1.000000
1  S000065  2000361487104  2000361487128  1.000000
2  S000065  2000361487104  2000361487135  0.913619
  hiercode       prodcode   prodcode_rel   sim_idx
0  S000065  2000361396857  2000361397052  1.000000
1  S000065  2000361396857  2000361396840  0.901469
2  S000065  2000361396857  2000361397045  0.901469
3  S000065  2000361396857  2000361396833  0.901469
4  S000065  2000361396857  2000361396826  0.901469
5  S000065  2000361396857  2000361397021  0.901469
6  S000065  2000361396857  2000361397038  0.901469
   hiercode       prodcode   prodcode_rel   sim_idx
0   S000065  2000361396956  2000361396864  0.654726
1   S000065  2000361396956  2000361396932  0.654726
2   S000065  2000361396956  2000361396727  0.654726
3   S000065  2000361396956  2000361396734  0.654726
4   S000065  2000361396956  2000361396741  0.654726
5   S000065  2000361396956  2000361396925  0.654726
6   S000065  20003613969

...

  hiercode       prodcode   prodcode_rel   sim_idx
0  S039004  2000356174422  2000356174392  0.836049
1  S039004  2000356174422  2000356174408  0.836049
2  S039004  2000356174422  2000356174415  0.836049
3  S039004  2000356174422  2000356174439  0.701702
4  S039004  2000356174422  2000356174446  0.696366
5  S039004  2000356174422  2000356174453  0.696351
   hiercode       prodcode   prodcode_rel   sim_idx
0   S039004  2000356175962  2000356175948  1.000000
1   S039004  2000356175962  2000356175955  1.000000
2   S039004  2000356175962  2000356175986  0.844600
3   S039004  2000356175962  2000356175979  0.843976
4   S039004  2000356175962  2000356175993  0.840298
5   S039004  2000356175962  2000356176006  0.840279
6   S039004  2000356175962  2000356175733  0.543016
7   S039004  2000356175962  2000356175740  0.543016
8   S039004  2000356175962  2000356175757  0.543016
9   S039004  2000356175962  2000356175894  0.539387
10  S039004  2000356175962  2000356175887  0.539387
11  S039004  200035

   hiercode       prodcode   prodcode_rel   sim_idx
0   S039004  2000361651154  2000361651178  1.000000
1   S039004  2000361651154  2000361651161  1.000000
2   S039004  2000361651154  2000361651192  0.908999
3   S039004  2000361651154  2000361651185  0.908866
4   S039004  2000361651154  2000357904066  0.512611
5   S039004  2000361651154  2000357904059  0.512611
6   S039004  2000361651154  2000357904042  0.512611
7   S039004  2000361651154  2000357904097  0.505534
8   S039004  2000361651154  2000357904103  0.505534
9   S039004  2000361651154  2000357904110  0.505534
10  S039004  2000361651154  2000357904011  0.503034
11  S039004  2000361651154  2000357904004  0.503034
12  S039004  2000361651154  2000357903991  0.503034
  hiercode       prodcode   prodcode_rel   sim_idx
0  S039004  2000361651970  2000361651932  0.945276
1  S039004  2000361651970  2000361651956  0.945276
2  S039004  2000361651970  2000361651949  0.945276
3  S039004  2000361651970  2000361651963  0.890466
4  S039004  20003

  hiercode       prodcode   prodcode_rel   sim_idx
0  S039004  2000361325888  2000361325895  1.000000
1  S039004  2000361325888  2000361325901  1.000000
2  S039004  2000361325888  2000361325925  0.949441
3  S039004  2000361325888  2000361325918  0.927649
   hiercode       prodcode   prodcode_rel   sim_idx
0   S039004  2000345605470  2000345605487  1.000000
1   S039004  2000345605470  2000345605494  1.000000
2   S039004  2000345605470  2000345605517  0.891709
3   S039004  2000345605470  2000345605500  0.891573
4   S039004  2000345605470  2000345605524  0.887243
5   S039004  2000345605470  2000345605531  0.887229
6   S039004  2000345605470  2000345605548  0.732136
7   S039004  2000345605470  2000345605555  0.732136
8   S039004  2000345605470  2000345605562  0.732136
9   S039004  2000345605470  2000345605685  0.707558
10  S039004  2000345605470  2000345605692  0.707558
11  S039004  2000345605470  2000345605708  0.707558
12  S039004  2000345605470  2000345605616  0.694878
13  S039004  2000

   hiercode       prodcode   prodcode_rel   sim_idx
0   S039004  2000346228296  2000346228302  1.000000
1   S039004  2000346228296  2000346228319  1.000000
2   S039004  2000346228296  2000346228326  0.867345
3   S039004  2000346228296  2000346228333  0.867297
4   S039004  2000346228296  2000346228340  0.858497
5   S039004  2000346228296  2000346228357  0.858481
6   S039004  2000346228296  2000346228364  0.630940
7   S039004  2000346228296  2000346228371  0.630940
8   S039004  2000346228296  2000346228388  0.630940
9   S039004  2000346228296  2000346228401  0.550516
10  S039004  2000346228296  2000346228395  0.550096
11  S039004  2000346228296  2000346228418  0.547174
12  S039004  2000346228296  2000346228425  0.547164
   hiercode       prodcode   prodcode_rel   sim_idx
0   S039004  2000346234563  2000346234570  1.000000
1   S039004  2000346234563  2000346234587  1.000000
2   S039004  2000346234563  2000346234600  0.878913
3   S039004  2000346234563  2000346234594  0.878724
4   S039004 

   hiercode       prodcode   prodcode_rel   sim_idx
0   S039004  2000340365584  2000340365591  1.000000
1   S039004  2000340365584  2000340365607  1.000000
2   S039004  2000340365584  2000340365621  0.893909
3   S039004  2000340365584  2000340365614  0.892829
4   S039004  2000340365584  2000340365638  0.888660
5   S039004  2000340365584  2000340365645  0.888645
6   S039004  2000340365584  2000340365720  0.696365
7   S039004  2000340365584  2000340365737  0.696365
8   S039004  2000340365584  2000340365744  0.696365
9   S039004  2000340365584  2000340365799  0.694686
10  S039004  2000340365584  2000340365805  0.694686
11  S039004  2000340365584  2000340365812  0.694686
12  S039004  2000340365584  2000340365867  0.694654
13  S039004  2000340365584  2000340365874  0.694654
14  S039004  2000340365584  2000340365881  0.694654
15  S039004  2000340365584  2000340365652  0.694266
16  S039004  2000340365584  2000340365669  0.694266
17  S039004  2000340365584  2000340365676  0.694266
18  S039004 

In [102]:
hiercode=lst_hiercode[0]
hiercode

'S000065'

In [103]:
dfiltered=df_input[df_input.hiercode==hiercode]
dfiltered=dfiltered[['prodcode','description']]

In [104]:
cosine_similarities_matrix=get_matrix(dfiltered)
 

In [105]:
cosine_similarities_matrix

array([[1.        , 0.14082434, 0.15512708, ..., 0.00861866, 0.00861866,
        0.00861866],
       [0.14082434, 1.        , 0.90779985, ..., 0.00802083, 0.00802083,
        0.00802083],
       [0.15512708, 0.90779985, 1.        , ..., 0.00883546, 0.00883546,
        0.00883546],
       ...,
       [0.00861866, 0.00802083, 0.00883546, ..., 1.        , 1.        ,
        1.        ],
       [0.00861866, 0.00802083, 0.00883546, ..., 1.        , 1.        ,
        1.        ],
       [0.00861866, 0.00802083, 0.00883546, ..., 1.        , 1.        ,
        1.        ]])

In [106]:
indeces=get_idx(dfiltered)

In [41]:
indeces

prodcode
2000362744275       0
2000362744794       1
2000362744787       2
2000362744770       3
2000362744763       4
                 ... 
2000369035246    4638
2000369035239    4639
2000369035222    4640
2000369035215    4641
2000369035208    4642
Length: 4643, dtype: int64

In [107]:
prodcode='2000362744275'

In [108]:
idx = indeces[prodcode]

In [109]:
idx

0

In [110]:
idx2=indeces['2000362744282']
print(idx2)

26


In [44]:
sim_scores = list(enumerate(cosine_similarities_matrix[idx]))


In [45]:
sim_scores

[(0, 0.9999999999999997),
 (1, 0.1408243432211539),
 (2, 0.15512708293101968),
 (3, 0.15512708293101968),
 (4, 0.15512708293101968),
 (5, 0.1408243432211539),
 (6, 0.15512708293101968),
 (7, 0.15512708293101968),
 (8, 0.15512708293101968),
 (9, 0.1408243432211539),
 (10, 0.15512708293101968),
 (11, 0.15512708293101968),
 (12, 0.1408243432211539),
 (13, 0.15512708293101968),
 (14, 0.35246368765757574),
 (15, 0.15512708293101968),
 (16, 0.35246368765757574),
 (17, 0.35246368765757574),
 (18, 0.35246368765757574),
 (19, 0.35246368765757574),
 (20, 0.15512708293101968),
 (21, 0.32133936079252573),
 (22, 0.35246368765757574),
 (23, 0.35246368765757574),
 (24, 0.35246368765757574),
 (25, 0.9116949406280747),
 (26, 0.9999999999999997),
 (27, 0.1408243432211539),
 (28, 0.9999999999999997),
 (29, 0.32133936079252573),
 (30, 0.35246368765757574),
 (31, 0.35246368765757574),
 (32, 0.35246368765757574),
 (33, 0.32133936079252573),
 (34, 0.35246368765757574),
 (35, 0.35246368765757574),
 (36, 0.352

In [46]:
sim_scores2 = list(enumerate(cosine_similarities_matrix[idx2]))

In [47]:
sim_scores2

[(0, 0.9999999999999997),
 (1, 0.1408243432211539),
 (2, 0.15512708293101968),
 (3, 0.15512708293101968),
 (4, 0.15512708293101968),
 (5, 0.1408243432211539),
 (6, 0.15512708293101968),
 (7, 0.15512708293101968),
 (8, 0.15512708293101968),
 (9, 0.1408243432211539),
 (10, 0.15512708293101968),
 (11, 0.15512708293101968),
 (12, 0.1408243432211539),
 (13, 0.15512708293101968),
 (14, 0.35246368765757574),
 (15, 0.15512708293101968),
 (16, 0.35246368765757574),
 (17, 0.35246368765757574),
 (18, 0.35246368765757574),
 (19, 0.35246368765757574),
 (20, 0.15512708293101968),
 (21, 0.32133936079252573),
 (22, 0.35246368765757574),
 (23, 0.35246368765757574),
 (24, 0.35246368765757574),
 (25, 0.9116949406280747),
 (26, 0.9999999999999997),
 (27, 0.1408243432211539),
 (28, 0.9999999999999997),
 (29, 0.32133936079252573),
 (30, 0.35246368765757574),
 (31, 0.35246368765757574),
 (32, 0.35246368765757574),
 (33, 0.32133936079252573),
 (34, 0.35246368765757574),
 (35, 0.35246368765757574),
 (36, 0.352

In [48]:
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
list_sim_scores=[list(element) for element in sim_scores]
list_sim_scores_filtered=list(filter(lambda x: x[0]!=idx,[list(element) for element in sim_scores]))

In [49]:
list_sim_scores_filtered

[[26, 0.9999999999999997],
 [28, 0.9999999999999997],
 [25, 0.9116949406280747],
 [14, 0.35246368765757574],
 [16, 0.35246368765757574],
 [17, 0.35246368765757574],
 [18, 0.35246368765757574],
 [19, 0.35246368765757574],
 [22, 0.35246368765757574],
 [23, 0.35246368765757574],
 [24, 0.35246368765757574],
 [30, 0.35246368765757574],
 [31, 0.35246368765757574],
 [32, 0.35246368765757574],
 [34, 0.35246368765757574],
 [35, 0.35246368765757574],
 [36, 0.35246368765757574],
 [38, 0.35246368765757574],
 [39, 0.35246368765757574],
 [40, 0.35246368765757574],
 [42, 0.35246368765757574],
 [45, 0.35246368765757574],
 [54, 0.35246368765757574],
 [59, 0.35246368765757574],
 [61, 0.35246368765757574],
 [62, 0.35246368765757574],
 [63, 0.35246368765757574],
 [64, 0.35246368765757574],
 [66, 0.35246368765757574],
 [67, 0.35246368765757574],
 [77, 0.35246368765757574],
 [81, 0.35246368765757574],
 [86, 0.35246368765757574],
 [21, 0.32133936079252573],
 [29, 0.32133936079252573],
 [33, 0.321339360792525

In [50]:
sim_scores_list=[  [hiercode,
                        prodcode,
                        indeces[indeces == list(element)[0]].index[0],
                        list(element)[1]] 
                        for element 
                        in list_sim_scores_filtered ]


In [51]:
sim_scores_list

[['S000065', '2000362744275', '2000362744282', 0.9999999999999997],
 ['S000065', '2000362744275', '2000362744268', 0.9999999999999997],
 ['S000065', '2000362744275', '2000362744299', 0.9116949406280747],
 ['S000065', '2000362744275', '2000362744480', 0.35246368765757574],
 ['S000065', '2000362744275', '2000362744374', 0.35246368765757574],
 ['S000065', '2000362744275', '2000362744367', 0.35246368765757574],
 ['S000065', '2000362744275', '2000362744473', 0.35246368765757574],
 ['S000065', '2000362744275', '2000362744466', 0.35246368765757574],
 ['S000065', '2000362744275', '2000362744336', 0.35246368765757574],
 ['S000065', '2000362744275', '2000362744329', 0.35246368765757574],
 ['S000065', '2000362744275', '2000362744312', 0.35246368765757574],
 ['S000065', '2000362744275', '2000362744237', 0.35246368765757574],
 ['S000065', '2000362744275', '2000362744220', 0.35246368765757574],
 ['S000065', '2000362744275', '2000362744213', 0.35246368765757574],
 ['S000065', '2000362744275', '200036

In [52]:
sim_scores_list_filtered=list(filter(lambda x:x[3]>=0.5,sim_scores_list))

In [53]:
sim_scores_list_filtered

[['S000065', '2000362744275', '2000362744282', 0.9999999999999997],
 ['S000065', '2000362744275', '2000362744268', 0.9999999999999997],
 ['S000065', '2000362744275', '2000362744299', 0.9116949406280747]]

In [54]:
df = pd.DataFrame(sim_scores_list_filtered)

In [56]:
df.columns=['hiercode','prodcode','prodcode_rel','sim_idx']
df

,hiercode,prodcode,prodcode_rel,sim_idx
0,S000065,2000362744275,2000362744282,1.000000
1,S000065,2000362744275,2000362744268,1.000000
2,S000065,2000362744275,2000362744299,0.911695


In [72]:
prod_list_exclude=[]

In [73]:
prod_rel_list=df['prodcode_rel'].tolist()

In [74]:
df['prodcode'][0]

'2000362744275'

In [75]:
prod_rel_list.append(df['prodcode'][0])

In [76]:
prod_list_exclude.extend(prod_rel_list)
prod_list_exclude

['2000362744282', '2000362744268', '2000362744299', '2000362744275']

In [77]:
lst=[x for x in list(dfiltered['prodcode']) if x not in  prod_list_exclude]

In [78]:
len(lst)

4639

In [23]:
list_sim_scores_filtered=[list(element) for element in sim_scores]

In [90]:
indeces['2000371553325']

1101

In [24]:
list_sim_scores_filtered

[[0, 0.9999999999999997],
 [1, 0.1408243432211539],
 [2, 0.15512708293101968],
 [3, 0.15512708293101968],
 [4, 0.15512708293101968],
 [5, 0.1408243432211539],
 [6, 0.15512708293101968],
 [7, 0.15512708293101968],
 [8, 0.15512708293101968],
 [9, 0.1408243432211539],
 [10, 0.15512708293101968],
 [11, 0.15512708293101968],
 [12, 0.1408243432211539],
 [13, 0.15512708293101968],
 [14, 0.35246368765757574],
 [15, 0.15512708293101968],
 [16, 0.35246368765757574],
 [17, 0.35246368765757574],
 [18, 0.35246368765757574],
 [19, 0.35246368765757574],
 [20, 0.15512708293101968],
 [21, 0.32133936079252573],
 [22, 0.35246368765757574],
 [23, 0.35246368765757574],
 [24, 0.35246368765757574],
 [25, 0.9116949406280747],
 [26, 0.9999999999999997],
 [27, 0.1408243432211539],
 [28, 0.9999999999999997],
 [29, 0.32133936079252573],
 [30, 0.35246368765757574],
 [31, 0.35246368765757574],
 [32, 0.35246368765757574],
 [33, 0.32133936079252573],
 [34, 0.35246368765757574],
 [35, 0.35246368765757574],
 [36, 0.352

In [25]:
sim_scores_list=[  [hiercode,
                        prodcode,
                        indeces[indeces == list(element)[0]].index[0],
                        list(element)[1]] 
                        for element 
                        in list_sim_scores_filtered ]

In [26]:
sim_scores_list

[['S000065', '2000362744275', '2000362744275', 0.9999999999999997],
 ['S000065', '2000362744275', '2000362744794', 0.1408243432211539],
 ['S000065', '2000362744275', '2000362744787', 0.15512708293101968],
 ['S000065', '2000362744275', '2000362744770', 0.15512708293101968],
 ['S000065', '2000362744275', '2000362744763', 0.15512708293101968],
 ['S000065', '2000362744275', '2000362744749', 0.1408243432211539],
 ['S000065', '2000362744275', '2000362744732', 0.15512708293101968],
 ['S000065', '2000362744275', '2000362744725', 0.15512708293101968],
 ['S000065', '2000362744275', '2000362744718', 0.15512708293101968],
 ['S000065', '2000362744275', '2000362744695', 0.1408243432211539],
 ['S000065', '2000362744275', '2000362744688', 0.15512708293101968],
 ['S000065', '2000362744275', '2000362744671', 0.15512708293101968],
 ['S000065', '2000362744275', '2000362744596', 0.1408243432211539],
 ['S000065', '2000362744275', '2000362744589', 0.15512708293101968],
 ['S000065', '2000362744275', '20003627

In [89]:
print(type(None))

<class 'NoneType'>
